In [1]:
"""Importación de las librerías necesarias"""

import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
import random
import numpy
from time import time

In [2]:
""" Variables de los tipos de casillas que existen en el tablero """ 

mina = '☠'
casilla_sin_pulsar = '□'

In [3]:
""" Datos de entrada """

filas = 5
columnas = 5
minas = 10

In [4]:
"""
    Función para crear un tablero destapado.
    Dependerá del número de filas, columnas y número de minas.
""" 
 
def tablero_buscaminas(filas, columnas, minas):
    # Comprobamos que el número de minas existente en el tablero es menor o igual al número de casillas del mismo
    if(minas > filas * columnas):
        raise Exception ("El número de minas debe ser menor o igual a" + filas*columnas)
        
    # Construcción del tablero
    tableroBuscaminas = [[0 for x in range(filas)] for y in range(columnas)]
    
    """
        Creamos minas aleatorias en el tablero. 
        Recorremos el número de minas que queremos pintar y en el tablero mostramos "☠" que indica que en esa casilla existe una mina.
    """
    
    for i in range(minas):
        tableroBuscaminas[random.randint(0,filas-1)][random.randint(0,columnas-1)] = mina
        
    # Ahora pasamos a poner los índices de las casillas colindantes con minas. 
    # El número que aparece en la casilla va en un rango de 1 a 8 (arriba, abajo, izquierda, derecha y las 4 diagonales).
    
    minas_colindantes = 0
    for fila in range(filas):
        for columna in range(columnas):
            if(fila > 0 and tableroBuscaminas[fila-1][columna] == mina): # La mina se encuentra arriba
                minas_colindantes += 1
            if(fila +1 < len(tableroBuscaminas) and tableroBuscaminas[fila+1][columna] == mina): # La mina se encuentra abajo
                minas_colindantes +=1
            if(columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila][columna+1] == mina):
                # La mina se encuentra a la derecha
                minas_colindantes +=1
            if(columna > 0 and tableroBuscaminas[fila][columna-1] == mina): # La mina se encuentra a la izquierda
                minas_colindantes +=1
            if(fila > 0 and columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila-1][columna+1] == mina):
                # La mina se encuentra en la diagonal superior derecha
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna+1 < len(tableroBuscaminas[0]) and 
                   tableroBuscaminas[fila+1][columna+1] == mina): 
                # La mina se encuentra en la diagonal inferior derecha
                minas_colindantes +=1
            if(fila > 0 and columna > 0 and tableroBuscaminas[fila-1][columna-1] == mina):
                # La mina se encuentra en la diagonal superior izquierda
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna > 0 and tableroBuscaminas[fila+1][columna-1] == mina): 
                # La mina se encuentra en la diagonal inferior izquierda
                minas_colindantes +=1
            # Pasamos a guardar el índice de minas colindantes en la casilla del tablero donde no hay ninguna mina.
            if(minas_colindantes > 0 and tableroBuscaminas[fila][columna] != mina ):
                tableroBuscaminas[fila][columna] = str(minas_colindantes)
            minas_colindantes=0
            
    return tableroBuscaminas

In [5]:
"""
    Prueba de creación del tablero con los parámetros indicados
"""
tableroBuscaminas = tablero_buscaminas(filas,columnas,minas)
print(numpy.array(tableroBuscaminas))

[['☠' '☠' '☠' '3' '1']
 ['2' '4' '☠' '☠' '2']
 ['0' '1' '3' '4' '☠']
 ['1' '1' '1' '☠' '2']
 ['☠' '1' '1' '1' '1']]


In [6]:
"""
    Función para generar un tablero tapado.
    Las casillas tapadas se mostrarán con el símbolo □
"""

def ocultar_casillas_tablero(filas,columnas):
    tablero_buscaminas_oculto = [[casilla_sin_pulsar for columna in range(columnas)] for fila in range(filas)]
        
    return tablero_buscaminas_oculto

In [7]:
""" Prueba de ocultación del tablero"""

tablero_buscaminas_oculto = ocultar_casillas_tablero(filas,columnas)
print(numpy.array(tablero_buscaminas_oculto))

[['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']]


In [8]:
"""
    Función para destapar una casilla del tablero.
    Al pulsar sobre una casilla que no contiene una mina, se van a destapar un
    conjunto colindante de casillas que no tienen minas ni están proximas con otras, es decir, con índice = 0. 
    Como máximo, el límite del destapado va a ser si la casilla que se destapa tiene al menos un vecino con mina.
"""

def destapar_casillas_tablero(fila,columna, tablero_buscaminas_oculto, tableroBuscaminas):
    
    # Guardo el valor de la casilla en la que pulsamos
    casillaDestapada = tableroBuscaminas[fila][columna]
    
    # Si la casilla pulsada es una mina:
    if(casillaDestapada == mina):
        print(numpy.array(tableroBuscaminas))
        raise Exception("Has encontrado una mina. Has perdido")
    # Si no lo es, cambiamos su valor en el tablero oculto por el valor de la casilla destapada
    else:
        tablero_buscaminas_oculto[fila][columna] = casillaDestapada
    
    """
        Comprobamos si la casilla que destapamos tiene 0 minas colindantes.
        Si se da el caso, destapamos todas las casillas colindantes a la seleccionada 
        ya que sabemos que ninguna es una mina se comprueba que las colindantes 
        estén dentro del tablero
    """
    
    if(casillaDestapada == 0): 
        if(fila > 0):
            destapar_casillas_tablero(fila-1,columna, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< len(tableroBuscaminas)):
            destapar_casillas_tablero(fila+1,columna, tablero_buscaminas_oculto, tableroBuscaminas)
        if((columna+1)< len(tableroBuscaminas[0])):
            destapar_casillas_tablero(fila,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if(columna > 0):
            destapar_casillas_tablero(fila,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)
        if(fila > 0 and (columna+1) < len(tableroBuscaminas[0])):
            destapar_casillas_tablero(fila-1,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< len(tableroBuscaminas) and (columna +1) < len(tableroBuscaminas[0])):
            destapar_casillas_tablero(fila+1,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if(fila > 0 and columna > 0):
            destapar_casillas_tablero(fila-1,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< len(tableroBuscaminas) and columna > 0):
            destapar_casillas_tablero(fila+1,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)
            

    return tablero_buscaminas_oculto


In [9]:
"""
    Prueba del destapado de una casilla.
"""

destapar_casillas_tablero(3,2, tablero_buscaminas_oculto, tableroBuscaminas)

[['□', '□', '□', '□', '□'],
 ['□', '□', '□', '□', '□'],
 ['□', '□', '□', '□', '□'],
 ['□', '□', '1', '□', '□'],
 ['□', '□', '□', '□', '□']]

In [10]:
"""
    Función que devuelve un array con los índices de las casillas vecinas a la que se le pasa por parámetros.
"""

def posicion_casillas_vecinas(fila,columna,tableroBuscaminas):

    indicesCasillas = []
    if((fila-1)>=0 and columna>=0):
        indicesCasillas.append("{0}{1}".format(fila-1, columna))
    if((fila+1)< len(tableroBuscaminas) and columna>=0):
        indicesCasillas.append("{0}{1}".format(fila+1, columna))
    if((fila>=0 and (columna+1) < len(tableroBuscaminas))):
        indicesCasillas.append("{0}{1}".format(fila, columna+1))
    if(fila>=0 and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila, columna-1))
    if(((fila-1)>=0 and (columna+1) < len(tableroBuscaminas))):
        indicesCasillas.append("{0}{1}".format(fila-1, columna+1))
    if((fila+1)< len(tableroBuscaminas) and (columna+1)< len(tableroBuscaminas)):
        indicesCasillas.append("{0}{1}".format(fila+1, columna+1))
    if((fila-1)>=0 and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila-1, columna-1))
    if((fila+1)< len(tableroBuscaminas) and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila+1, columna-1))

    return indicesCasillas

In [11]:
"""
    Prueba de la función anterior
"""

posicion_casillas_vecinas(0,1,tableroBuscaminas)

['11', '02', '00', '12', '10']

In [12]:
"""
    Función que, dado un tablero, devuelve el listado de minas que se encuentran en él.
"""

def listado_de_minas(tableroBuscaminas):
    listado_minas = [] # Vamos a obtener las casillas donde están las minas situadas
    for fila in range(filas):
        for columna in range(columnas):
            if(tableroBuscaminas[fila][columna] == mina):
                listado_minas.append('Tapada({0}{1})'.format(fila,columna))
    return listado_minas


In [13]:
listado_de_minas(tableroBuscaminas)

['Tapada(00)',
 'Tapada(01)',
 'Tapada(02)',
 'Tapada(12)',
 'Tapada(13)',
 'Tapada(24)',
 'Tapada(33)',
 'Tapada(40)']

In [14]:
Minas_encontradas = listado_de_minas(tableroBuscaminas)

In [15]:
# Creación de la red bayesiana
def red_bayesiana(filas,columnas,tablero_buscaminas_oculto):
    """
        Creamos la red bayesiana con la librería pgmpy
    """
    Buscaminas_bayesiano = pgmm.BayesianModel()
    
    """
        Variables importantes:
        · x_totales: casillas ocultas totales del tablero
        · x_ocultas: casillas ocultas con vecinos destapados
        · y_destapadas: casillas destapadas
    """
    
    start_time = time()
    
    x_totales = []
    x_ocultas = []
    y_destapadas = []
    casilla_oculta_sin_colindantes_destapados = False
    
    """
        Añadimos los nodos a la red.
        Serán las casillas ocultas con vecinos destapados y las casillas destapadas.
    """
    
    for fila in range(filas):
        for columna in range(columnas):
            
            # Obtenemos la posicion de las casillas vecinas
            casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas)
            
            # Si la casilla no está pulsada
            if (tablero_buscaminas_oculto[fila][columna] == casilla_sin_pulsar):
                
                x = 'Tapada({0}{1})'.format(fila,columna) # Lo pone en formato Tapada (01),Tapada (02)...
                x_totales.append(x) # Añadimos las casillas sin pulsar a la variable x_totales
                casilla_tiene_vecinos_destapados = False
                
                # Recorro todas las casillas vecinas de la casilla en la que estamos actualmente
                for vecino in casillas_vecinas:
                    # Comprobamos que los vecinos de la casilla estan destapados
                    if(tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] != casilla_sin_pulsar):
                        casilla_tiene_vecinos_destapados = True
                        break
                 # Comprobamos que la casilla actual tiene vecino destapado
                if(not casilla_oculta_sin_colindantes_destapados or casilla_tiene_vecinos_destapados):
                    # Si lo tiene, añadimos la casilla a la lista de casillas tapadas con vecinos destapados
                    x_ocultas.append(x)
                    Buscaminas_bayesiano.add_nodes_from([x]) # Se añade la casilla como nodo de nuestra red

                if(not casilla_tiene_vecinos_destapados):
                    casilla_oculta_sin_colidantes_destapados = True
            
            # Si la casilla está pulsada
            else:
                y = 'Destapada({0}{1})'.format(fila,columna) # lo pone en formato Destapada (01),Destapada (02)...
                casilla_tiene_vecinos_ocultos = False
                #Listado_de_minas = listado_de_minas(tableroBuscaminas) # Variable que contiene la posición de las minas del tablero
                
                for vecino in casillas_vecinas:
                    casilla_vecina_oculta = 'Tapada({0}{1})'.format(vecino[0],vecino[1])
                    if(tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] == casilla_sin_pulsar and
                       not Minas_encontradas.__contains__(casilla_vecina_oculta)):
                        casilla_tiene_vecinos_ocultos = True
                        break

                if(casilla_tiene_vecinos_ocultos):
                    y_destapadas.append(y)
                    Buscaminas_bayesiano.add_nodes_from([y])

    print("Lista Completa casillas ocultas : \n")
    print(x_totales)
    print("\n")
    print("Lista casillas vecinas ocultas acumuladas hasta la posicion actual : \n")
    print(x_ocultas)
    print("\n")
    print("Lista casillas destapadas hasta ese momento : \n")
    print(y_destapadas)
    print("\n")

    print("Comprobamos la lista de nodos añadidos a la red bayesiana:\n")
    print(Buscaminas_bayesiano.nodes())
    print("\n")

    print("Lista de casillas ocultas que son minas:\n")
    print(Minas_encontradas)
    print("\n")

    """
        Añadimos las aristas entre los nodos.
        Las casillas destapadas tendrán aristas con sus vecinas ocultas
    """

    Aristas = dict()
    ListaColindantesOcultos = []
    for fila in range(filas):
        for columna in range(columnas):
            Destapada = 'Destapada({0}{1})'.format(fila,columna)
            
            # Si la casilla ha sido destapada y se ha añadido a la lista de casillas destapadas
            if(tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar
                  and y_destapadas.__contains__(Destapada)):
                
                # Obtenemos la posicion de las casillas vecinas
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas)
                
                # Recorremos sus vecinos
                for vecino in casillas_vecinas: # Recorro sus vecinos
                    Tapada = 'Tapada({0}{1})'.format(vecino[0],vecino[1])
                    
                    # Si el vecino es oculto añadimos la arista de la destapada a la tapada
                    if(x_ocultas.__contains__(Tapada)):
                        Buscaminas_bayesiano.add_edges_from([(Tapada,Destapada)])
                        ListaColindantesOcultos.append(Tapada)
                        
                # Asociamos la casilla destapada con todos sus vecinos ocultos  
                Aristas.update({Destapada: ListaColindantesOcultos})

    print("Lista de aristas:\n")
    print(Aristas)
    print("\n")
    print("Lista de vecinos ocultos:\n")
    print(ListaColindantesOcultos)
    print("\n")
    print("Lista de aristas añadidas a la red bayesiana:\n") 
    print(Buscaminas_bayesiano.edges())
    print("\n")

    """
        Generación de tablas de probabilidad (CPDs)
        · Las tablas de probabilidad de las casillas tapadas no dependen de nada.
        · Las tablas de probabilidad de las casillas destapadas dependen de sus casillas vecinas tapadas.
    """
    
    """
        Para cada variable creamos una instancia de la clase `TabularCPD`, proporcionando:
        · Nombre de la variable (atributo `variable`),
        · Cantidad de valores (cardinalidad) que puede tomar (atributo `variable_card`),
        · Lista de listas, conteniendo cada una de estas las probabilidades para un valor concreto, 
          según los valores de los padres (atributo `values`, el valor se transforma a un array),
        · Lista con los nombres de los padres (atributo `evidence`, valor `None` si no se proporciona),
        · Lista con la cantidad de valores (cardinalidad) que puede tomar cada uno de los padres (atributo `evidence_card`, 
          valor `None` si no se proporciona).
    """
    
    """
        Tabla de probabilidad de las casillas tapadas
    """

    ListaValoresDestapados=dict()
    for fila in range(filas):
        for columna in range(columnas):
            Tapada = 'Tapada({0}{1})'.format(fila,columna)
            Destapada = 'Destapada({0}{1})'.format(fila,columna)
            if(tablero_buscaminas_oculto[fila][columna]==casilla_sin_pulsar and x_ocultas.__contains__(Tapada)):
                
                # Si la casilla por detapar es una mina
                if(Minas_encontradas.__contains__(Tapada)):
                    probabilidad_mina = 1

                # Si el numero de casillas por destapar es mayor que el numero de minas que tiene el tablero
                elif(len(x_ocultas)>0):
                    #numero de casillas destapadas = (X*Y)-len(x_ocultas)
                    probabilidad_mina = (minas-len(Minas_encontradas))/len(x_totales)
                else:
                    probabilidad_mina = 0

                # Añadimos la cpd al modelo
                cpd = pgmf.TabularCPD(Tapada, 2, [[1-probabilidad_mina,probabilidad_mina]])
                Buscaminas_bayesiano.add_cpds(cpd)


            # CPD de Destapadas
            elif(tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar and y_destapadas.__contains__(Destapada)):
                ListaValoresDestapados.update({Destapada: int(tablero_buscaminas_oculto[fila][columna])})

               #A continuación divido entre los valores que puede tener 'Destapada' en función de las minas que tenga alrededor
                nColumnasCPD = pow(2, len(Aristas[Destapada]))
                numero_maximo_minas = 9

                # Si no toca ninguna pared como máximo 8 minas
                # Si toca un lado como máximo 5 minas
                # Si toca 2 lados (está en una esquina) como máximo 3 minas
                # if((i == 0 and j == 0) or (i==X and j == Y) or (i == 0 and j == Y) or (i==X and j ==0)): #esto es una esquina
                #    numeroMaximoMinas = 3
                # if((i == 0 and (j > 0 and j < Y)) or (j == 0 and (i > 0 and i < X)) or (i==X and (j > 0 and j < Y)) or (j==Y and (i > 0 and i < X))): #esto es 1 lado
                #    numeroMaximoMinas = 5
                # else:
                #    numeroMaximoMinas = 8


                if(fila==0 or fila==filas-1):
                    numero_maximo_minas = 6
                if(columna==0 or columna==columnas-1):
                    if(numero_maximo_minas == 6):
                        numero_maximo_minas = 4
                    else:
                        numero_maximo_minas = 6
                if(len(Aristas[Destapada])+1 < numero_maximo_minas):
                    numero_maximo_minas = len(Aristas[Destapada])+1

                listaProb = []
                for numero_minas in range(numero_maximo_minas):
                    contador = 0
                    listaFila = []
                    for x in range(nColumnasCPD):
                        if(numero_minas == bin (contador).count('1')):
                            listaFila.append(1)
                        else:
                            listaFila.append(0)
                        contador = contador + 1

                    listaProb.append(listaFila)

                ListaCardinalidad = []
                for _ in range(len(Aristas[Destapada])):
                    ListaCardinalidad.append(2)

                cpd = pgmf.TabularCPD(Destapada, numero_maximo_minas, listaProb, Aristas[Destapada], ListaCardinalidad)

                Buscaminas_bayesiano.add_cpds(cpd)

                print("\nIMPRIMO TODOS LOS FACTORES DE LA CPD: \n")

                #Cada columna arriba, indica los padres de la variable que se calcula la probabilidad.
                for m in Buscaminas_bayesiano.cpds: print(m)

    Buscaminas_bayesiano_ev = pgmi.VariableElimination(Buscaminas_bayesiano) 
    consultas = []
    for variable in x_ocultas:
        if(not variable in Minas_encontradas):
            consulta = Buscaminas_bayesiano_ev.query([variable], ListaValoresDestapados)
            consultas.append(consulta)
            
    ocultasProbabilidadDeNoSerMina = dict()
    for variable in consultas:
        x = ''
        probabilidadDeNoSerMina = 0
        for variable1 in variable.keys():
            x = str(variable1)
        for variable2 in variable.values():
            probabilidadDeNoSerMina = float(variable2.values[0])
            
        if(probabilidadDeNoSerMina == 0.0):
            Minas_encontradas.append(x)
            
        
        ocultasProbabilidadDeNoSerMina.update({x: probabilidadDeNoSerMina}) 
        
    maximaProbabilidadDeNoSerMina = 0.
    ocultaMaximaProbabilidadDeNoSerMina = ''
    mejor_oculta = []
    print("Mejor casilla oculta con alta probabilidad de no ser mina:\n")
    for x, probabilidadDeNoSerMina in ocultasProbabilidadDeNoSerMina.items():
        print("x: {0} -> Probabilidad de no ser mina: {1}".format(x, probabilidadDeNoSerMina))
        if (maximaProbabilidadDeNoSerMina < probabilidadDeNoSerMina):
            ocultaMaximaProbabilidadDeNoSerMina = str(x)
            maximaProbabilidadDeNoSerMina = probabilidadDeNoSerMina
            mejor_oculta = []
            mejor_oculta.append(ocultaMaximaProbabilidadDeNoSerMina)
        if (maximaProbabilidadDeNoSerMina == probabilidadDeNoSerMina and not x in mejor_oculta):
            mejor_oculta.append(x)
        if (probabilidadDeNoSerMina == 1.0 and not x in mejor_oculta):
            mejor_oculta.append(x)
        
        

    # Refreshing list of best_x to not be mine
    print("\nLista de mejores casillas ocultas con probabilidad de no ser minas: ")
    print("\n{0}".format(mejor_oculta))
    finish_time = time() 
    print("\n Tiempo de ejecucción:\n")
    print(finish_time - start_time)
    return Buscaminas_bayesiano

In [16]:
print(red_bayesiana(filas, columnas, tablero_buscaminas_oculto))

Lista Completa casillas ocultas : 

['Tapada(00)', 'Tapada(01)', 'Tapada(02)', 'Tapada(03)', 'Tapada(04)', 'Tapada(10)', 'Tapada(11)', 'Tapada(12)', 'Tapada(13)', 'Tapada(14)', 'Tapada(20)', 'Tapada(21)', 'Tapada(22)', 'Tapada(23)', 'Tapada(24)', 'Tapada(30)', 'Tapada(31)', 'Tapada(33)', 'Tapada(34)', 'Tapada(40)', 'Tapada(41)', 'Tapada(42)', 'Tapada(43)', 'Tapada(44)']


Lista casillas vecinas ocultas acumuladas hasta la posicion actual : 

['Tapada(00)', 'Tapada(01)', 'Tapada(02)', 'Tapada(03)', 'Tapada(04)', 'Tapada(10)', 'Tapada(11)', 'Tapada(12)', 'Tapada(13)', 'Tapada(14)', 'Tapada(20)', 'Tapada(21)', 'Tapada(22)', 'Tapada(23)', 'Tapada(24)', 'Tapada(30)', 'Tapada(31)', 'Tapada(33)', 'Tapada(34)', 'Tapada(40)', 'Tapada(41)', 'Tapada(42)', 'Tapada(43)', 'Tapada(44)']


Lista casillas destapadas hasta ese momento : 

['Destapada(32)']


Comprobamos la lista de nodos añadidos a la red bayesiana:

['Tapada(00)', 'Tapada(01)', 'Tapada(02)', 'Tapada(03)', 'Tapada(04)', 'Tapada(10)', 'T

Mejor casilla oculta con alta probabilidad de no ser mina:

x: Tapada(03) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(04) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(10) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(11) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(14) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(20) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(21) -> Probabilidad de no ser mina: 1.0
x: Tapada(22) -> Probabilidad de no ser mina: 1.0
x: Tapada(23) -> Probabilidad de no ser mina: 1.0
x: Tapada(30) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(31) -> Probabilidad de no ser mina: 1.0
x: Tapada(34) -> Probabilidad de no ser mina: 0.9166666666666666
x: Tapada(41) -> Probabilidad de no ser mina: 1.0
x: Tapada(42) -> Probabilidad de no ser mina: 1.0
x: Tapada(43) -> Probabilidad de no ser mina: 1.0
x: Tapada(44) -> Probabilidad de no ser mina: 0.9166666666666666

List